In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import json
import random

In [2]:
# Dataframe for hour, name, avg_rating, enrollment, difficulty, link, site 

def coursera_scraper (keyword):
    
    keyword = re.sub(" ","%20",keyword)
    url = f"https://www.coursera.org/search?query={keyword}&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English"
    page = list(range(2,30))
    urls = []
    urls.append(url)
    for i in page:
        urls.append("https://www.coursera.org/search?query=" + keyword + "&page="+ str(i) + "&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English")


    hour = []
    name = []
    avg_rating = []
    enrollment = []
    difficulty = []
    link = []
    site = []

    for url in urls:
        try:
            website_url = requests.get(url, \
                                  headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            soup_html = BeautifulSoup(website_url, "lxml")
            courses = soup_html.findAll("script")
            html = courses[4]
            json_object = json.loads(html.contents[0])
            t =  json_object["props"]["pageProps"]["resultsState"]
            for i in t:
                x = i['content']["_rawResults"][0]["hits"]
                for j in x:
                    if("avgLearningHours" in j):
                        hour.append(j["avgLearningHours"])
                        name.append(j["name"])
                        avg_rating.append(j["avgProductRating"])
                        enrollment.append(j["enrollments"])
                        difficulty.append(j["productDifficultyLevel"])
                        link.append('https://www.coursera.org' + j["objectUrl"])
                        site.append('cousera')
            
            time.sleep(random.randint(5,15))
            print(url)
        except Exception as e:
            print (e)

    course_df = pd.DataFrame({'hour': hour, 'name': name, 'avg_rating': avg_rating, 'enrollment': enrollment, 
                              'difficulty': difficulty, 'link': link, 'site': site})

    return course_df

In [8]:
## Dataframe for duration and review
def duration_reviews (dataframe):  
    duration = []
    reviews = []
    
    for url in dataframe['link']:
        try:

            website_url = requests.get(url, \
                                      headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            soup_html = BeautifulSoup(website_url, "lxml")
            
            # duration
            text = soup_html.find_all('div', {"class":"_16ni8zai m-b-0"})
            time_frame = text[4].getText()
            length = re.findall('[\d]{1,2} \S+', time_frame)
            duration.append(length)
            
            # review
            revs = soup_html.find_all('p', {"class":"rc-TopReviewsListItem__comment"})
            review = ''
            for i in revs:
                review += str(i.getText())
            reviews.append(review)
            
            # sleep
            time.sleep(random.randint(5,10))
            print(url)

        except Exception as e:
            duration.append('None indicated')
            reviews.append('None indicated')
            print (e)
    
    return pd.DataFrame({'duration': duration, 'review': reviews})

In [4]:
def make_csv(keyword):
    # Input: keyword is string 
    df = coursera_scraper(keyword)
    dur_rev =  duration_reviews(df)
    combined_df = pd.concat([df, dur_rev], axis = 1)
    combined_df.to_csv('coursera_' + str(keyword) + '.csv')
    return combined_df

In [ ]:
# SAMPLE SCRAPING CODE

make_csv('Mathematics')